### Réseau de neurones basé sur iDeep

In [ ]:
import keras
from keras.models import Sequential,load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.layers import Merge
#from keras.optimizers import kl_divergence

In [ ]:
from sklearn import svm, grid_search
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.cross_validation import train_test_split, StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_curve
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib 
from scipy import sparse

In [ ]:
import numpy as np
import random
import gzip
import pdb
from math import  sqrt
#import theano
import subprocess as sp
import scipy.stats as stats
import argparse

In [ ]:
def get_cnn_network_microRNA():    
    print('configure cnn network')
    nbfilter = 22

    model = Sequential()
    model.add(Conv1D(activation="relu", input_shape=(25, 4), filters=22, kernel_size=7, strides=1, padding="valid"))

    model.add(MaxPooling1D(pool_size=3))
    
    model.add(Dropout(0.3))
    
    model.add(Flatten())
    
    model.add(Dense(nbfilter, activation='relu'))
    model.add(Dropout(0.2))
    #model.add(Activation('relu'))
    #model.add(PReLU())
    #model.add(BatchNormalization(mode=2))
    #model.add(Dense(64))
 
    #model.fit(X_train, y_train)
    
    return model

In [ ]:
def get_cnn_network_messengerRNA():    
    print('configure cnn network')
    nbfilter = 100

    model = Sequential()
    model.add(Conv1D(activation="relu", input_shape=(101, 4), filters=100, kernel_size=7, strides=1, padding="valid"))

    model.add(MaxPooling1D(pool_size=3))
    
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dropout(0.25))
    
    #model.add(Dense(nbfilter, activation='relu'))
    #model.add(Activation('relu'))
    #model.add(PReLU())
    #model.add(BatchNormalization(mode=2))
    #model.add(Dense(64))    
    
    #model.fit(X_train, y_train)
    
    return model

In [ ]:
def cnn2D():
    nb_conv = 4
    nb_pool = 2
    model = Sequential()
    model.add(Conv2D(64, (nb_conv, nb_conv), padding='valid', input_shape=(1, 101,4),strides=(1,1)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    model.add(Dropout(0.25))
    model.add(Flatten())

In [ ]:
def get_mlp_microRNA(num_hidden = 64):
    model = Sequential()

    #model.add(Dense(num_hidden, input_dim=train.shape[1], activation='relu'))
    model.add(Dense(num_hidden, input_shape=(25,), activation='relu'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(num_hidden, input_dim=num_hidden, activation='relu'))
    #model.add(Dense(num_hidden, input_shape=(num_hidden,), activation='relu'))
    model.add(PReLU())
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(Dropout(0.5))
    '''
    model.add(Dense(sec_num_hidden, input_shape=(num_hidden,), activation='relu'))
    model.add(PReLU())
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(Dropout(0.5))
    '''
    return model

In [ ]:
def get_mlp_messengerRNA(num_hidden = 128):
    model = Sequential()

    #model.add(Dense(num_hidden, input_dim=train.shape[1], activation='relu'))
    model.add(Dense(num_hidden, input_shape=(101,), activation='relu'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_hidden, input_dim=num_hidden, activation='relu'))
    #model.add(Dense(num_hidden, input_shape=(num_hidden,), activation='relu'))
    model.add(PReLU())
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(Dropout(0.5))
    '''
    model.add(Dense(sec_num_hidden, input_shape=(num_hidden,), activation='relu'))
    model.add(PReLU())
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(Dropout(0.5))
    '''
    return model

In [ ]:
def calculate_performance(test_num, pred_y,  labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1               
            
    acc = float(tp + tn)/test_num
    precision = float(tp)/(tp+ fp)
    sensitivity = float(tp)/ (tp+fn)
    specificity = float(tn)/(tn + fp)
    MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    return acc, precision, sensitivity, specificity, MCC 


In [ ]:
def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[2:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

verif = pd.read_csv("negatifs.csv", sep = "\t",header=None)
verif = np.array(verif)
for i in range(len(verif)):
    for j in range(4):
        if len(verif[i,j]) <= 15:
            print(i,"erreur")
            

for i in range(len(verif)):
    verif[i,2] = np.array(clean(verif[i,2]))
    verif[i,3] = np.array(clean(verif[i,3]))    

neg = verif

verif = pd.read_csv("positifs.csv", sep = "\t",header=None)  
verif = np.array(verif)
for i in range(len(verif)):
    for j in range(4):
        if len(verif[i,j]) <= 15:
            print(i,"erreur")
            

for i in range(len(verif)):
    verif[i,2] = np.array(clean(verif[i,2]))
    verif[i,3] = np.array(clean(verif[i,3]))

pos = verif
verif = []
bdd = np.concatenate((pos,neg))
pos = []
neg = []
labels = np.zeros((len(bdd),1))

bdd = np.concatenate((bdd,labels),axis=1)
for i in range(int(len(bdd)/2)):
    bdd[i,4]=1    
labels=[]
for i in range(len(bdd)):
    bdd[i,0] = transformer_sequence(bdd[i,0],len(bdd[i,0]))
    bdd[i,1] = transformer_sequence(bdd[i,1],101)
    bdd[i,2] = transformer_structure(bdd[i,2])

# shuffle pour mélanger positifs et négatifs

indices = np.zeros(len(bdd),dtype=int)
for i in range(int(len(bdd)/2)):
    indices[2*i] = int(i)
    indices[2*i+1] = int(i + int(len(bdd)/2))
bdd = bdd[indices]
indices = []

# shuffle total

indices = np.arange(len(bdd))
shuffle(indices)
bdd = bdd[indices]

for i in range(len(bdd)):
    bdd[i,0] = bdd[i,0].transpose()
    bdd[i,1] = bdd[i,1].transpose()

resh0 = np.zeros((20048,25,4))

for i in range(len(bdd)):
    resh0[i,:,:] = bdd[i,0]
    
resh1 = np.zeros((20048,101,4))

for i in range(len(bdd)):
    resh1[i,:,:] = bdd[i,1]

resh2 = np.zeros((20048,25))

for i in range(len(bdd)):
    resh2[i,:] = bdd[i,2]
    
resh3 = np.zeros((20048,101))

for i in range(len(bdd)):
    resh3[i,:] = bdd[i,3]

In [ ]:
nb_train = 17000
nb_val = 500
nb_test = 2500

In [ ]:
resh0_train = resh0[:nb_train]
resh0_val = resh0[nb_train:nb_train+nb_val]
resh1_train = resh1[:nb_train]
resh1_val = resh1[nb_train:nb_train+nb_val]
resh2_train = resh2[:nb_train]
resh2_val = resh2[nb_train:nb_train+nb_val]
resh3_train = resh3[:nb_train]
resh3_val = resh3[nb_train:nb_train+nb_val]
resh0_test = resh0[-nb_test:]
resh1_test = resh1[-nb_test:]
resh2_test = resh2[-nb_test:]
resh3_test = resh3[-nb_test:]
resh0=[]
resh1=[]
resh2=[]
resh3=[]
train = bdd[:nb_train]
valid = bdd[nb_train:nb_train+nb_val]
y = train[:,4]
y = keras.utils.np_utils.to_categorical(y,2)
val_y = valid[:,4]
val_y = keras.utils.np_utils.to_categorical(val_y,2)
train=[]
valid=[]
test = bdd[-nb_test:]
bdd = []

In [ ]:
def merge_networks_train_predict():
    
    print('training', nb_train)
    
    micro_structure_hid = 64
    messenger_structure_hid = 128
    micro_seq_hid = 22
    messenger_seq_hid = 100
    
    micro_seq_train = resh0_train
    micro_seq_validation = resh0_val
    micro_seq_net =  get_cnn_network_microRNA()
    
    messenger_seq_train = resh1_train
    messenger_seq_validation = resh1_val
    messenger_seq_net = get_cnn_network_messengerRNA()
    
    micro_structure_train = resh2_train
    micro_structure_validation = resh2_val
    micro_structure_net = get_mlp_microRNA()
    
    messenger_structure_train = resh3_train
    messenger_structure_validation = resh3_val
    messenger_structure_net = get_mlp_messengerRNA()        
    
    #y, encoder = preprocess_labels(training_label)
    #val_y, encoder = preprocess_labels(validation_label, encoder = encoder)
       
    training_net=[]
    training = []
    validation = []
    total_hid = 0

    print("Création des réseaux pour les deux séquences")

    training_net.append(micro_seq_net)
    training.append(micro_seq_train)
    validation.append(micro_seq_validation)
    total_hid = total_hid + micro_seq_hid
    micro_seq_train = []
    micro_seq_validation = [] 
    
    training_net.append(messenger_seq_net)
    training.append(messenger_seq_train)
    validation.append(messenger_seq_validation)
    total_hid = total_hid + messenger_seq_hid
    messenger_seq_train = []
    messenger_seq_validation = []
    
    print("Concaténation des deux séquences")
    
    model1 = Sequential()
    model1.add(Merge(training_net, mode='concat'))
    model1.add(Dropout(0.5))
    print(total_hid)
    model1.add(Dense(122, input_shape=(122,)))
    model1.add(Activation('softmax'))
    
    print("Création des réseaux pour les deux structures")
    
    training_net=[]
    
    training_net.append(micro_structure_net)
    training.append(micro_structure_train)
    validation.append(micro_structure_validation)
    total_hid = total_hid + micro_structure_hid
    micro_structure_train = []
    micro_structure_validation = []
    
    training_net.append(messenger_structure_net)
    training.append(messenger_structure_train)
    validation.append(messenger_structure_validation)
    total_hid = total_hid + messenger_structure_hid
    messenger_structure_train = []
    messenger_structure_validation = []
    
    print("Concaténation des deux structures")
    
    model2 = Sequential()
    model2.add(Merge(training_net, mode='concat'))
    model2.add(Dropout(0.5))
    print(total_hid)
    model2.add(Dense(192, input_shape=(192,)))
    model2.add(Activation('softmax'))
    
    print("Concaténation des deux modèles")
    
    training_net=[]
    training_net.append(model1)
    training_net.append(model2)
    
    model = Sequential()
    model.add(Merge(training_net, mode='concat'))
 
    #model.add(Dense(total_hid, input_shape=(total_hid,)))
    #model.add(Activation('relu'))
    #model.add(PReLU())
    #model.add(BatchNormalization(mode=2))
    #model.add(Activation('relu'))
    
    model.add(Dropout(0.5))
    print(total_hid)
    model.add(Dense(2, input_shape=(total_hid,)))
    model.add(Activation('softmax'))
    
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    
    #checkpointer = ModelCheckpoint(filepath="models/bestmodel.hdf5", verbose=0, save_best_only=True)
    earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=0)
    #validation_data=(np.transpose(validmat['validxdata'],axes=(0,2,1)), validmat['validdata']), callbacks=[checkpointer,earlystopper]
    print('model training')
    model.fit(training, y, batch_size=10000, epochs=1000, verbose=0, validation_data=(validation, val_y), callbacks=[earlystopper])
    
    training = []
    validation = []
    
    # test
    true_y = test[:,4]
    
    print('predicting')
    testing = []
    testing.append(resh0_test)
    testing.append(resh1_test)
    testing.append(resh2_test)
    testing.append(resh3_test)
        
    predictions = model.predict_proba(testing)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true_y)
    perfs = calculate_performance(len(predictions), predictions, true_y)
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])

    
    return model

In [ ]:
m = merge_networks_train_predict()

##### 100 trains : 
acc :  0.5
precision :  0.5333333333333333
sensitivity :  0.7272727272727273
specificity :  0.2222222222222222
MCC :  -0.058025885318565944

##### 17000 trains
acc :  0.5964
precision :  0.6069017254313578
sensitivity :  0.625193199381762
specificity :  0.5655058043117744
MCC :  0.19100235122777048
##### 
acc :  0.5564
precision :  0.5346628679962013
sensitivity :  0.8972111553784861
specificity :  0.21285140562248997
MCC :  0.15103195995226454
(10000 batch et 1000 epochs)